# Geo spatial Retrieval Augmented Generation using Hybrid search retrieval using Elastic, Anthropic Claude 3, Amazon Bedrock and Langchain

## Introduction

In this notebook we will show you how to use Elastic search, Amazon Bedrock, Anthropic Claude 3 and Langchain to build a Retrieval Augmented Generation (RAG) solution that leverages Geospatial features of Elastic.

### Use case
This notebook introduces an interactive AI agent designed to assist with real estate inquiries. Users looking to purchase properties, such as townhomes, condos, or single-family homes, in specific locations (e.g., New York, NY or Cupertino, CA) can use the agent to find listings that match their preferences. These preferences can include factors like city, distance, property type, and desired amenities such as a swimming pool or lawn.

### Implementation
In this notebook, we'll demonstrate how to build a geospatial RAG (Retrieval-Augmented Generation) application using Elastic's geospatial capabilities in combination with LLMs from the Amazon Bedrock platform. The tools we'll use include Elasticsearch, the Anthropic Claude 3 Sonnet Foundation model, Amazon Bedrock, and Langchain. Additionally, we'll integrate Amazon Location Service for geocoding physical addresses and Amazon Simple Email Service (SES) to send property listing recommendations to users.


#### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠


## Installation

To run this notebook you would need to install dependencies - boto3, botocore, elasticsearch and langchain.

In [ ]:
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain --force-reinstall --quiet
%pip install langchain_community --force-reinstall --quiet
%pip install langchain-elasticsearch --force-reinstall --quiet
%pip install elasticsearch --force-reinstall --quiet
%pip install unstructured==0.7.12 --force-reinstall --quiet
%pip install prettytable --force-reinstall --quiet

## Kernel Restart

Restart the kernel with the updated packages that are installed through the dependencies above

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Troubleshooting Flag

Use this `debug` flag and turn it to `True` if you would like to see more troubleshooting verbose generated. Otherwise, leave it as `False` value.

In [ ]:
debug = False;

## Setup 

Import the necessary libraries

In [3]:
import json
import os
import sys
import boto3
import botocore
import nltk
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.bedrock import BedrockChat
from langchain.embeddings import BedrockEmbeddings
from botocore.client import Config
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_elasticsearch import ElasticsearchStore
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import RetrievalQA
from getpass import getpass
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader
from pathlib import Path
from typing import Dict
from langchain_elasticsearch import ElasticsearchRetriever

## Geospatial Data indexing in to Elastic

### Connecting to Elastic Endpoints

We'll use the Cloud ID to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to [Cloud ID](https://cloud.elastic.co/deployments) and select your deployment.

We will use Elasticsearch to connect to our elastic cloud deployment. This would help create and index data easily. 

In [19]:
from elasticsearch import Elasticsearch

cloud_id = getpass("Elastic deployment Cloud ID: ")
cloud_api_key = getpass("Elastic deployment API Key: ")
index_name = 'geo_spatial_test3'  # Choose an appropriate index name


els_client = Elasticsearch(
    cloud_id=cloud_id,
    api_key=cloud_api_key
)

Elastic deployment Cloud ID:  ········
Elastic deployment API Key:  ········


### Real Estate Property Data
`data.json` file has all of the data related to Real Estate properties. The data is fabricated and created for demo purposes. While the geographic coordinates factually exists and is real, the rest of the data is purely synthetic and created for the learning purposes. 

Currently the data is created for the following cities:
- Frisco, TX
- Cupertino, CA

Property Types are:
- Townhomes
- Condos
- Multi Family
- Single Family Residence


In [76]:
FILE = "data.json"


metadata_keys = ['propertyId','propertyName' ,'propertyAddress', 'propertyCity', 'propertyState', 'propertyZip', 'propertyType', 'propertyCoordinates', 'propertyFeatures']
documents = []
with open(FILE, 'rt') as f:
    for doc in json.loads(f.read()):
        metadata={k: doc.get(k) for k in metadata_keys}        
        action = {
           "_index": index_name,
           "_source": metadata
        }
        documents.append(action)

### Data Schema
The following mappings define the data model for the data this will be indexed in Elasticsearch. Notice that `propertyCoordinates` is of type `geo_point` which indicates that it will store the geographic coordinates in the form of `longitude` and `latitude`.

In [77]:
mapping = {
  "mappings": {
    "properties": {
      "propertyName": {
        "type": "text"
      },
      "propertyAddress": {
        "type": "text"
      },
      "propertyCity": {
        "type": "text"
      },
      "propertyCoordinates": {
        "type": "geo_point"
      },
      "propertyFeatures": {
        "type": "text"
      },
      "propertyId": {
        "type": "long"
      },
      "propertyState": {
        "type": "text"
      },
      "propertyType": {
        "type": "text"
      },
      "propertyZip": {
        "type": "text"
      }
    }
  }
}

### Indexing
First we will delete the index and data to create a clean slate to load data afresh.
Next we will create the index and bulk load all of the data.

In [78]:
# Delete the index first
els_client.indices.delete(index=index_name, ignore_unavailable=True)

ObjectApiResponse({'acknowledged': True})

In [ ]:
# Create the index with the mapping
els_client.indices.create(index=index_name, body=mapping, ignore=400)

In [80]:
#Perform bulk indexing
success, failure = bulk(els_client, documents)

## Initialization

Initiate Bedrock Runtime and BedrockChat

In [81]:
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')

modelId = 'anthropic.claude-instant-v1' # change this to use a different version from the model provider

embeddingmodelId = 'amazon.titan-embed-text-v1' # change this to use a different embedding model

llm = BedrockChat(model_id=modelId, client=bedrock_client)
embeddings = BedrockEmbeddings(model_id=embeddingmodelId,client=bedrock_client)

## Entity Extraction from the prompt

Here we will do the extraction of :
* address
* property type
* radius within which the customer is looking for a property
* additional property features that the customer is looking for (optional)

from the user prompt.

In [82]:
user_prompt_template = """
Human: You will be acting as a Real estate realtor. Your end users will ask questions about finding properties near to a specific location given in the form of an address. This address may sometimes contains the Street number, Street Name, City name, State name and zip code. If the country name is not mentioned, consider it as USA.  For the prompt, extract the longitude and latitude information by geocoding.

For example, for the prompt below:

<prompt>
Get me townhomes within 5 miles near 2379 Flicker Street, Frisco, TX 75034. I want the townhome to have a swimming pool in the backyard and kitchen bigger.
</prompt>

Based on the above prompt, the property type the user is looking for is : townhomes. Return this as search_property_type in the JSON output.
address is : 2379 Flicker Street, Frisco, TX 75034. Return this as search_property_address in the JSON output.
radius is : 5mi
Return this as search_property_radius in the JSON output.
property features are :  townhome to have a swimming pool in the backyard and a bigger kitchen. 
Return this as search_property_features in the JSON output.

The address component may or may not have all the street, city , state or zipcode portion. Extract whatever is available.
If the property type is not mentioned, default the value as "Single Family Residence".
If the property type is mentioned anything like house or single family or residence, then emit the value as "Single Family Residence".
If the property type is mentioned anything close to town home or townhouse etc, then emit the value as "Townhouse".
If the property type is mentioned anything close to multi family or duplex or multiple family, then emit the value as "Multi Family".
If the property type is mentioned anything like Condomonium or Condo or condos etc, then emit the value as "Condos".
If the property type value is not determinable always default the value as "Single Family Residence".

The radius component may be expressed in miles or in kilometers. Output should in the format like 5mi or 5km depending upon the unit of measure used - miles or kilometers. If no radius component is found, default it to 6mi.
If property features are not found, return it as 'Nil'.
Return these entities in a JSON output format as a python JSON variable I can use. Do not output any other verbose.

Here is the user’s question: <question> {question} </question>

How do you respond to the user’s question?
Think about your answer first before you respond. Give your response in JSON format and exclude any other additional verbose.
Assistant:
"""

In [104]:
def extract_entities(question):
    prompt_template = PromptTemplate(template=user_prompt_template, input_variables=["question"])
    prompt = prompt_template.format (question=question)

    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 3000})
    accept = 'application/json'
    contentType = 'application/json'
    extracted_entities = {}

    try:
        response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read())

        # print(response_body.get('completion'))
        extracted_entities = response_body.get('completion')
        print("*** Extracted Entities from the prompt are: ***")
        print(extracted_entities)

    except botocore.exceptions.ClientError as error:

        if error.response["Error"]["Code"] == "AccessDeniedException":
            print(
                f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n"
            )

        else:
            raise error

    #qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=retriever, return_source_documents=True, chain_type_kwargs={"prompt": prompt})

    return extracted_entities

## AWS Location Services for Geocoding
Here we will leverage AWS location services for geocoding. 
Geocoding is the process of converting addresses (like a street address) into geographic coordinates (latitude and longitude), which can be used to place markers on a map or identify locations in spatial data. It helps map a physical location, such as "1600 Pennsylvania Ave NW, Washington, D.C.," into its corresponding geographic coordinates, enabling applications like GPS navigation, location-based services, or geographic information systems (GIS).

### Why are we geocoding?
The purpose of geocoding is to convert the extracted geographical location from the user prompt in to longitude and latitude, so that these coordinates can be used to search for real estate properties data in Elastic.

In [85]:
# Initialize the AWS Location Service client
location_client = boto3.client('location')

def invoke_aws_loc_service(address, index_name='explore.place.Esri'):
    
    if address is None or address == "":
        print("Address is None or empty")
        return None

    try:
        # Call the search_place_index_for_text method
        response = location_client.search_place_index_for_text(
            IndexName=index_name,
            Text=address
        )

        # Extract the first result (assuming it's the most relevant)
        if response['Results']:
            place = response['Results'][0]['Place']
            longitude, latitude = place['Geometry']['Point']
            
            return {
                'address': address,
                'longitude': longitude,
                'latitude': latitude,
                'label': place.get('Label', ''),
                'country': place.get('Country', '')
            }
        else:
            return None

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


## Run Geospatial Elastic Query

Now that we geocoded the search location, we will pass that to the elastic query, to search for properties that existing within a distance radius of user prefered location.  We will also pass the search property type (like townhome, single family residence etc) to the query so that we find the exact property that the user is looking for. 

In [113]:
def run_elastic_geospatial_query (geo_coded_lat, geo_coded_long, search_property_radius,search_property_type, index_name):
    resp = els_client.search (
        index=index_name,
        query= {
            "bool": {
                "must": [
                    { "match": { "propertyType": search_property_type }}
                ],
                "filter": [
                    {
                        "geo_distance" : {
                            "distance": search_property_radius, "propertyCoordinates": {"lat": geo_coded_lat, "lon": geo_coded_long }
                        }
                    }
                ]
            }
        }
    )
    return resp


## Geospatial RAG in action

The real estate properties data found from Elastic is now passed as an additional context to the LLM via Amazon Bedrock, to perform RAG. 

In [107]:
def run_geospatial_rag(question, context):
    user_recommendation_template = """
Human: You will be acting as a Real estate realtor. Your end users will ask questions about finding properties near to a specific location given in the form of an address. This address may sometimes contains the Street number, Street Name, City name, State name and zip code. If the country name is not mentioned, consider it as USA.  

You will answer only based on the context given: <context>{context}</context>
Here are some important rules for the interaction:
- Always stay in character of being a Real estate advisor, trying to help find properties.
- If you are unsure how to respond, say “Sorry, I didn’t understand that. Could you repeat the question?”
- If someone asks something irrelevant, say, “Sorry, I don't know.”

Here is the user’s question: <question> {question} </question>
Go ahead and answer.
Assistant:
    """

    recommendation_template = PromptTemplate(template=user_recommendation_template, input_variables=["context", "question"])
    prompt = recommendation_template.format (question=question,context=context)
    
    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 10000})
    accept = 'application/json'
    contentType = 'application/json'
    extracted_entities = {}

    try:
        response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read())

        if debug:
            print(response_body.get('completion'))
        return response_body.get('completion')
        

    except botocore.exceptions.ClientError as error:

        if error.response["Error"]["Code"] == "AccessDeniedException":
            print(
                f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n"
            )

        else:
            raise error


## Display the results from Elastic in a readable Tabular form

The following utility will display the results from Elastic in a tabular format.

In [115]:
from prettytable import PrettyTable

def display_pretty_table(results) :
    data = results
    
    # Create a PrettyTable object
    table = PrettyTable()

    # Define the columns
    table.field_names = ["Property ID","Property Name", "Address", "Type"]

    # Add rows to the table
    for hit in data['hits']['hits']:
        source = hit['_source']
        table.add_row([
            source['propertyId'],
            source['propertyName'],            
            source['propertyAddress'],
            source['propertyType'],
        ])

    # Set the alignment of the columns
    table.align["Address"] = "l"  # Left align the Address column
    table.align["Type"] = "l"     # Left align the Type column
    
    # Print the table
    print(table)


## Plotting Real Estate property details on a Map.

In order to visually show the geographic locations of each property that is found by searching Elastic database, we need to plot these on a Map.  The following `display_map` utility does the same.

In [84]:
import folium
from IPython.display import IFrame, display, HTML

def display_map(search_location, data):    
    points = []
    
    for hit in data['hits']['hits']:
        source = hit['_source']
        coordinates = source['propertyCoordinates'].split(',')
        latitude = float(coordinates[0])
        longitude = float(coordinates[1])
        new_point = {
            "name": source['propertyId'],
            "tooltip": source['propertyType'] + " : " + source['propertyName'] + " : " + source['propertyAddress'],
            "latitude": latitude,
            "longitude": longitude
        }
        points.append(new_point)
    

    # Calculate the center point
    if points:
        if debug:
            print ("Hurrayyyy , it has has some points added")
    else:
        Title = "<h2>Currently no properties found. Please check back with us. We are continuosly adding newer listings.</h2>"
        display(HTML(Title))
        return
        
    center_lat = sum(point['latitude'] for point in points) / len(points)
    center_lon = sum(point['longitude'] for point in points) / len(points)

    # Create a map centered on the mean of all points
    m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

    # Add markers for each point
    for point in points:
        folium.Marker(
            location=[point['latitude'], point['longitude']],
            popup=point['name'],
            tooltip=point['tooltip']
        ).add_to(m)

    # Save the map
    map_file = "results_map.html"
    m.save(map_file)

    # Display the map in the notebook
    Title = "<h2>Real Estate Properties near : " + search_location + "</h2>"
    display(HTML(Title))
    display(IFrame(src=map_file, width=800, height=800))

## Automated Email

Automagically send an email to the end user, so that they have all the property listings and recommendations in an email.

In [108]:
def send_email(sender, recipient, subject, body_html):
    # Replace with your AWS region
    AWS_REGION = "us-east-1"
    
    # Create a new SES resource and specify a region.
    client = boto3.client('ses', region_name=AWS_REGION)
    
    # Try to send the email.
    try:
        response = client.send_email(
            Destination={
                'ToAddresses': [
                    recipient,
                ],
            },
            Message={
                'Body': {
                    'Html': {
                        'Charset': 'UTF-8',
                        'Data': body_html,
                    },
                },
                'Subject': {
                    'Charset': 'UTF-8',
                    'Data': subject,
                },
            },
            Source=sender,
        )
    except ClientError as e:
        print(f"An error occurred: {e.response['Error']['Message']}")
    else:
        print(f"Email sent! Message ID: {response['MessageId']}")

## Lets put everything to work now.. Interactive Chat AI in action

In [ ]:
## how to ask a question
def ask_a_question(question):
    print(f'The most relevant question is : \n\t{question}')
    # Step 1 : Entitiy Extraction: Get the address, radius, property features from the search.
    extracted_entities_JSON = json.loads(extract_entities(question))
    search_property_type = extracted_entities_JSON["search_property_type"]
    search_property_address = extracted_entities_JSON["search_property_address"]
    search_property_radius = extracted_entities_JSON["search_property_radius"]
    search_property_features = extracted_entities_JSON["search_property_features"]
        
    # Step 2 : Geocoding using AWS Location Services. Using the address information, geocode and get longitude and latitude.
    aws_location_service_result = invoke_aws_loc_service(search_property_address)
    if (debug):
        print(aws_location_service_result)
        print(type(aws_location_service_result))

    geo_coded_long = aws_location_service_result["longitude"]
    geo_coded_lat = aws_location_service_result["latitude"]
    geo_coded_add = aws_location_service_result["address"]
    
    if (debug):
        print(f'And here is geocoded long for search address: {geo_coded_long}')
        print(f'And here is geocoded lat for search address: {geo_coded_lat}')
        print(f'And here is geocoded address for search address: {geo_coded_add}')
    
    # Step 3: Using the address coordinates and the radius, perform an Elastic search operation to find the nearest points of interest
    results = run_elastic_geospatial_query (geo_coded_lat, geo_coded_long, search_property_radius, search_property_type, index_name)
    data = results.body
    if debug:
        print(results.body)
    
    results_found = False
    
    for hit in data['hits']['hits']:
        source = hit['_source']
        results_found = True
    
    if results_found is False:
        Title = "<h2>Currently no properties found. Please check back with us. We are continuosly adding newer listings.</h2>"
        display(HTML(Title))
        return

    # Step 4: Run Geo Spatial RAG
    
    print("****************** Response - START **********************")
    rag_summary = run_geospatial_rag(question, results.body)
    print(rag_summary)
    
    # Step 5: Display results in a Tabular form
    display_pretty_table(results.body)
    
    # Step 6: Now plot the top 3 relevant search results on a Map.
    display_map(search_property_address, results.body)
    
    #Step 7: Send a personalized email to the Customer
    sender = "sripendi@amazon.com"
    recipient = "sripendi@amazon.com"
    subject = "Hello from AnyCompany Real Estate Management"
    body_html = """
    <html>
    <head></head>
    <body>
      <h1>Your interest in buying properties in {location} </h1>
      <p>Hello Customer,</p>
      <p>We are delighted to serve you.</p>
      <p>{rag_summary}</p>
      <p> Thankyou </p>
      <h3>AnyCompany Real Estate Management LLC Team </h3>
      <h4>Call us : 1.800.AAA.BBBB </h4>
    </body>
    </html>
    """.format(location=search_property_address, rag_summary=rag_summary)

    body_html = body_html

    if debug: 
        print(body_html)
    #send_email(sender, recipient, subject, body_html)
    
    print("****************** Response - END   **********************")
# The conversational loop

print("""

  __          __  _                                  _                               
 \ \        / / | |                                | |                              
  \ \  /\  / /__| | ___ ___  _ __ ___   ___        | |_ ___                         
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \       | __/ _ \                        
    \  /\  /  __/ | (_| (_) | | | | | |  __/       | || (_) |                       
  ___\/  \/ \___|_|\___\___/|_| |_| |_|\___|        \__\___/ _           _   _      
 |  _ \         | |              | |                 |  ____| |         | | (_)     
 | |_) | ___  __| |_ __ ___   ___| | __    ______    | |__  | | __ _ ___| |_ _  ___ 
 |  _ < / _ \/ _` | '__/ _ \ / __| |/ /   |______|   |  __| | |/ _` / __| __| |/ __|
 | |_) |  __/ (_| | | | (_) | (__|   <               | |____| | (_| \__ \ |_| | (__ 
 |____/_\___|\__,_|_|__\___/ \___|_|\_\   _       _  |______|_|\__,_|___/\__|_|\___|
  / ____|          / ____|           | | (_)     | |    |  __ \     /\   / ____|    
 | |  __  ___  ___| (___  _ __   __ _| |_ _  __ _| |    | |__) |   /  \ | |  __     
 | | |_ |/ _ \/ _ \\___ \| '_ \ / _` | __| |/ _` | |    |  _  /   / /\ \| | |_ |    
 | |__| |  __/ (_) |___) | |_) | (_| | |_| | (_| | |    | | \ \  / ____ \ |__| |    
  \_____|\___|\___/_____/| .__/ \__,_|\__|_|\__,_|_|    |_|  \_\/_/    \_\_____|    
                         | |                                                        
                         |_|                                                                                                        

""")

print(f'I am a trivia chat bot, ask me any question about RealEstate')

while True:
    question = input("User Question >> ")
    response= ask_a_question(question)
    print(f"\tAnswer  : {response}")



  __          __  _                                  _                               
 \ \        / / | |                                | |                              
  \ \  /\  / /__| | ___ ___  _ __ ___   ___        | |_ ___                         
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \       | __/ _ \                        
    \  /\  /  __/ | (_| (_) | | | | | |  __/       | || (_) |                       
  ___\/  \/ \___|_|\___\___/|_| |_| |_|\___|        \__\___/ _           _   _      
 |  _ \         | |              | |                 |  ____| |         | | (_)     
 | |_) | ___  __| |_ __ ___   ___| | __    ______    | |__  | | __ _ ___| |_ _  ___ 
 |  _ < / _ \/ _` | '__/ _ \ / __| |/ /   |______|   |  __| | |/ _` / __| __| |/ __|
 | |_) |  __/ (_| | | | (_) | (__|   <               | |____| | (_| \__ \ |_| | (__ 
 |____/_\___|\__,_|_|__\___/ \___|_|\_\   _       _  |______|_|\__,_|___/\__|_|\___|
  / ____|          / ____|           | | (_)     | |    |  __ 

User Question >>  Find me townhomes near Frisco, TX within 5 miles that is under $400K and has a big kitchen


The most relevant question is : 
	Find me townhomes near Frisco, TX within 5 miles that is under $400K and has a big kitchen
*** Extracted Entities from the prompt are: ***
 {
  "search_property_type": "Townhouse",
  "search_property_address": "Frisco, TX",
  "search_property_radius": "5mi", 
  "search_property_features": "under $400K and has a big kitchen"
}
****************** Response - START **********************
 Certainly, let me see what options we have that meet your criteria of a townhome near Frisco, TX within 5 miles, under $400K, and with a big kitchen:

It looks like there are a couple great possibilities! The first is 8410 Hickory St Unit 2103 in Frisco. This townhome is only about 1 mile from the center of Frisco and is currently listed at $269K. It has a gorgeous upgraded kitchen with lots of cabinets and granite countertops. The kitchen is open to the living area as well. This home backs up to a community green space for nice views. 

Another option is 8209 Canal St in

****************** Response - END   **********************
	Answer  : None
